# find the intersections within of an AOI against the tile db

In [ ]:
import xarray as xr
import hvplot.xarray
import dask
from dask.distributed import Client, progress
#from dask_kubernetes import KubeCluster
import os
import fsspec

In [ ]:
import pandas as pd
df = pd.read_pickle('../00-ET-inputs-and-outputs/tile_catalog_df.pkl')

In [ ]:
df

In [ ]:
import geopandas
from shapely.geometry import box
#import shapely.bounding.box as box

#Once you have converted the bounding boxes to polygons, make sure to actually create a GeoDataFrame:
b = df.apply(lambda row: box(row.left, row.bottom, row.right, row.top), axis=1)
gdf = geopandas.GeoDataFrame(df, geometry=b)

In [ ]:
gdf.head()

In [ ]:
import folium

tony_plot_tiles_json = gdf.to_json()

map_osm = folium.Map(location=[35, -90], zoom_start=5)

tile_boxes=folium.features.GeoJson(tony_plot_tiles_json)
map_osm.add_child(tile_boxes)

import json 

with open("atlanta_aoi.geojson") as f:
    atlanta_geo = json.load(f)

at = folium.features.GeoJson(atlanta_geo)
map_osm.add_child(at)
map_osm

In [ ]:
import json
from shapely.geometry import shape, GeometryCollection

with open("atlanta_aoi.geojson") as f:
  features = json.load(f)["features"]

# NOTE: buffer(0) is a trick for fixing scenarios where polygons have overlapping coordinates 
atlanta_shapely = GeometryCollection([shape(feature["geometry"]).buffer(0) for feature in features])

In [ ]:

new_gdf = gdf.loc[gdf.within(atlanta_shapely)]

In [ ]:
new_gdf

In [ ]:
new_gdf = gdf.loc[gdf.intersects(atlanta_shapely)]

In [ ]:
new_gdf

In [ ]:
rf_list=[]
for index,row in new_gdf.iterrows():
    print(row.Chip)
    #raster_file = '/vsis3/' + row.Chip + '/etasw_2014.tif'
    raster_file = row.Chip + '/etasw_2014.tif'
    rf_list.append(raster_file)

In [ ]:
rf_list

In [ ]:
from pangeoLib .aws_authenticate import aws_authenticate
aws_authenticate()

In [ ]:
da = xr.open_rasterio('s3://'+rf_list[0])
da
da = da.squeeze().drop(labels='band')
da.plot()

In [ ]:
ds = da.to_dataset(name='etasw_annual')


ds.etasw_annual.plot()

In [ ]:
ds

In [ ]:
ds.hvplot(grid=True)

In [ ]:
from time import time


In [ ]:
%%time
tifs = rf_list

start = time()
my_da_list =[]
for tif in tifs:
    da = xr.open_rasterio('s3://'+tif)
    da = da.squeeze().drop(labels='band')
    da.name='mosaic'
    my_da_list.append(da)
    tnow = time()
    elapsed = tnow - start
    print(tif, elapsed)
    
da

In [ ]:
ds = xr.merge(my_da_list)


In [ ]:
ds_masked = ds.where(ds>0.0)

In [ ]:
ds_masked.mosaic.plot(cmap='viridis')

In [ ]:
%%time
ds_masked.hvplot(grid=True, invert=True, width=600, height=500, cmap='viridis')

In [ ]:
ds_masked

In [ ]:
dir(ds)

In [ ]:
dir(gdf)